In [1]:
# Use

In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm, tqdm_pandas, tqdm_notebook
import os


In [2]:
tqdm.pandas()

In [3]:
!ls | grep bios


3-cy-get-yale-bios-zoo.ipynb
6-cy-get-harvard-bios-zoo.ipynb
7-cy-parse-yale-bios.ipynb
8-cy-parse-harvard-bios.ipynb
9-cy-parse-yale-bios-spark.ipynb


In [8]:

df = pd.read_csv('../data/raw/yale_bios.csv.gz', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8618 entries, 0 to 8617
Data columns (total 2 columns):
Unnamed: 0    8618 non-null float64
0             8535 non-null object
dtypes: float64(1), object(1)
memory usage: 134.7+ KB


In [6]:
df.head()

,Unnamed: 0,0
0,0.0,"<div class=""bio-wrap clearfix"" data-module=""st..."
1,1.0,"<div class=""bio-wrap clearfix"" data-module=""st..."
2,10.0,"<div class=""bio-wrap clearfix"" data-module=""st..."
3,100.0,"<div class=""bio-wrap clearfix"" data-module=""st..."
4,1000.0,"<div class=""bio-wrap clearfix"" data-module=""st..."


We'll need to break up the strings into blocks and only use the markup that we actually need.

In [7]:
# the output needs to be handled 1 at a time, and then dumped out carefully.
# df['soup'] = df['0'].progress_apply(lambda x: BeautifulSoup(x, 'lxml') )

In [8]:
# tSoup = df['0'].head().progress_apply(lambda x: BeautifulSoup(x, 'lxml') )

100%|██████████| 5/5 [00:00<00:00, 11.98it/s]


In [9]:
# ts = tSoup.iloc[0]

In [9]:
def tableToDict(soup):
    "Given a headerless table, return a dict with all the fields"
    table = {}
    for row in soup.find_all('tr'):
        cells = row.find_all('td')
        table[str(cells[0].text.strip())] = cells[1].text.strip()
        
    return table
    
def soupToDict(soup):
    "Given a player's soup blob, parse their header"
    playerData = tableToDict(soup.select('div.player-info')[0].find('table'))
    playerData['name'] = soup.select('div.player-name span.name')[0].text.strip()
    return playerData
def getSynopsis(soup):
    "Given a player's soup, split their bio information into \
    chunks for each strong header"
    
    # Get all top level children
#     bioPs = soup.select('div.synopsis')[0].find_all('p', recursive=False)
    
#     blocks= []
#     # if their blocks are malformed, skip that block.
#     for i, val in enumerate(bioPs):
#         try:
#             blocks[val.select('strong')[0].text] = val.get_text()
#         except:
#             printf("Text not detected")
        
    
    # Given the variation in blob types, just return
    # single blob for now.
    # Ryan Brenner: b instead of strong tags
    # Jackson Stallings (Junior Year) - UL blobs mixed with P tags
    # Jackson Stallings (Freshman) - Clean example
    
    # Sometimes, no bio
#     http://yalebulldogs.com/sports/c-sail/2016-17/bios/buehler_patrick_nu3o?view=news
    bio = soup.find('div',class_='synopsis')
    return bio.get_text() if bio else None

In [11]:
# ts.select('div.synopsis')[0].find_all('b')

[<b>2009:</b>,
 <b>Stuper Says:</b>,
 <b><u>Getting to Know Ryan</u></b>,
 <b>Parents' Names:</b>,
 <b>Major:</b>,
 <b>Best Athletic Moment:</b>,
 <b>Favorite MLB Team:</b>,
 <b>Favorite MLB Player:</b>,
 <b>Favorite Quote:</b>,
 <b>One word to describe yourself as a player:</b>,
 <b>Why Yale?:</b>,
 <b>Year       Avg  GP-GS    AB    R    H  2B  3B  HR  RBI   TB  SLG%  BB HBP  SO GDP   OB%  SF  SH  SB-ATT   PO   A   E  FLD%</b>,
 <b>TOTAL...  .211  22-20    57   12   12   6   0   1    7   21  .368   6  10  10   1  .384   0   0   1-2    108  10   3  .975</b>]

In [12]:
getSynopsis(ts)

u'\n2009: Made 22 appearances, including 20 starts...Started 18 games behind the plate, batting .211 with one home run and 12 RBI...Was hit by a pitch 10 times to lead the team...Went 2-for-5 with two doubles, an RBI and two runs scored on March 9 at Maryland-Baltimore County...Hit a two-run home run, the first of his career, in the March 17 win at Towson...Was a perfect 3-for-3 with two doubles and an RBI at Cornell on March 31...Went 3-for-4, scoring two runs, in game two of an April 19 doubleheader against Dartmouth...Drove in two runs with a double on April 26 at Brown.\nStuper Says: "Ryan will log quite a few innings in quite a few places. His best position is behind the plate, but he will see time at third, DH, and in the outfield . He is an outstanding hitter already with a very solid approach. I expect he will be in the lineup right away."\nGetting to Know Ryan\nParents\' Names: Marc and Suzanne Brenner\nMajor: Undeclared\nBest Athletic Moment: Winning the league championship m

In [13]:
soupToDict(ts)

{'Birth Date:': u'09/26/1990',
 'Height:': u'5-11',
 'High School:': u'Chaminade',
 'Hometown:': u'Manhasset, N.Y.',
 'Position:': u'C',
 'Weight:': u'180',
 'Year:': u'Freshman',
 'name': u'Ryan Brenner'}

In [14]:
# df['types'] = df['0'].map(type)
# df['types'].describe()

In [15]:
# df['types']

In [10]:
df['headers'] = df['0'].progress_apply(lambda x: \
                                       soupToDict(BeautifulSoup(x, 'lxml')) if (type(x) == str) else None)

100%|█| 8618/8618 [02:22<00:00, 60.45it/s]


In [12]:
import json

In [13]:
df['headers2'] = df['headers'].progress_apply(lambda x: json.dumps(x, encoding="utf-8"))


100%|█| 8618/8618 [00:00<00:00, 48689.25it/s]

In [14]:
df['headers'] = df['headers2']

In [15]:
df['synopsis'] = df['0'].progress_apply(lambda x: 0 )


100%|█| 8618/8618 [00:00<00:00, 374695.62it/s]

In [ ]:
# df['synopsis'] = df['0'].progress_apply(lambda x: getSynopsis(BeautifulSoup(x, 'lxml')) if (type(x) == str) else None)

In [21]:
linkSeries = pd.read_csv('../data/raw/yale_bio_urls.csv', header=None)

In [22]:
linkSeries.head()

,0
0,/sports/m-basebl/mtt/brenner_ryan00.html
1,/sports/m-basebl/mtt/castaldi_joe00.html
2,/sports/m-basebl/mtt/ludwig_pat00.html
3,/sports/m-basebl/2016-17/bios/cecere_bobby_a3p9
4,/sports/m-crewlt/2015-16/bios/sanderson_eric_edr3


In [23]:
df.index = linkSeries[0]

In [31]:
df.head()

,Unnamed: 0,0,headers,synopsis
0,,,,
/sports/m-basebl/mtt/brenner_ryan00.html,0.0,"<div class=""bio-wrap clearfix"" data-module=""st...","{u'Position:': u'C', u'Height:': u'5-11', u'Ye...","\n2009: Made 22 appearances, including 20 star..."
/sports/m-basebl/mtt/castaldi_joe00.html,1.0,"<div class=""bio-wrap clearfix"" data-module=""st...","{u'Position:': u'P', u'Height:': u'6-2', u'Yea...",\n2010: Team captain.\n2009: Made 12 appearanc...
/sports/m-basebl/mtt/ludwig_pat00.html,10.0,"<div class=""bio-wrap clearfix"" data-module=""st...","{u'Hometown:': u'Princeton, N.J.', u'Major:': ...",\n2015-16 (Junior): ThreeÂ seat of the fourth ...
/sports/m-basebl/2016-17/bios/cecere_bobby_a3p9,100.0,"<div class=""bio-wrap clearfix"" data-module=""st...","{u'name': u'Robert Wulbern', u'Year:': u'Jr.',...",\n2012-13: Rowed in stroke-seat of the 2V at E...
/sports/m-crewlt/2015-16/bios/sanderson_eric_edr3,1000.0,"<div class=""bio-wrap clearfix"" data-module=""st...","{u'Position:': u'DB', u'Year:': u'Senior', u'H...",\n2006 Backup DB who played in all but one gam...


In [17]:
df[['synopsis', 'headers2']].to_csv('../data/interim/yale_bio_parsed.csv.gz', compression='gzip')

In [34]:
!gzip -c yale_bio_parsed.csv > yale_bio_parsed.csv.gz

In [18]:
!ls -lS | grep csv.gz

-rw-r--r-- 1 cyick 197609 231002 Dec  2 18:53 yale_bio_parsed.csv.gz
